In [1]:
import os
import json

import numpy as np
import pandas as pd

from ipywidgets import Dropdown

from bqplot import Lines, Figure, LinearScale, DateScale, Axis

from ipyleaflet import Map, GeoJSON, WidgetControl


In [2]:
data = pd.read_json(os.path.abspath("nations.json"))

In [3]:
def clean_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data


def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.0)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range


def extrap_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data[column] = data[column].apply(extrap_interp)
    return data


In [4]:
data = clean_data(data)
data = extrap_data(data)

In [5]:
data

,name,region,income,population,lifeExpectancy
0,Angola,Sub-Saharan Africa,"[359.93, 359.93, 359.93, 359.93, 359.93, 359.9...","[1567028.0, 1567028.0, 1567028.0, 1567028.0, 1...","[26.98, 26.98, 26.98, 26.98, 26.98, 26.98, 26...."
1,Benin,Sub-Saharan Africa,"[553.72, 553.72, 553.72, 553.72, 553.72, 553.7...","[636559.0, 636559.0, 636559.0, 636559.0, 63655...","[31.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31...."
2,Botswana,Sub-Saharan Africa,"[407.36, 407.36, 407.36, 407.36, 407.36, 407.3...","[121000.0, 121000.0, 121000.0, 121000.0, 12100...","[33.6, 33.6, 33.6, 33.6, 33.6, 33.6, 33.6, 33...."
3,Burkina Faso,Sub-Saharan Africa,"[454.33, 454.33, 454.33, 454.33, 454.33, 454.3...","[1665421.0, 1665421.0, 1665421.0, 1665421.0, 1...","[29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29...."
4,Burundi,Sub-Saharan Africa,"[447.59, 447.59, 447.59, 447.59, 447.59, 447.5...","[899097.0, 899097.0, 899097.0, 899097.0, 89909...","[31.5, 31.5, 31.5, 31.5, 31.5, 31.5, 31.5, 31...."
...,...,...,...,...,...
174,Thailand,East Asia & Pacific,"[496.98, 496.98, 496.98, 496.98, 496.98, 496.9...","[4665000.0, 4665000.0, 4665000.0, 4665000.0, 4...","[30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30...."
175,Timor-Leste,East Asia & Pacific,"[514.12, 514.3505, 514.581, 514.8115, 515.042,...","[137262.0, 137262.0, 137262.0, 137262.0, 13726...","[28.97, 28.97, 28.97, 28.97, 28.97, 28.97, 28...."
177,Tonga,East Asia & Pacific,"[667.71, 667.71, 667.71, 667.71, 667.71, 667.7...","[18658.0, 18654.325581395347, 18650.6511627907...","[57.91, 57.91, 57.91, 57.91, 57.91, 57.91, 57...."
178,Vietnam,East Asia & Pacific,"[459.71, 459.71, 459.71, 459.71, 459.71, 459.7...","[6551000.0, 6551000.0, 6551000.0, 6551000.0, 6...","[32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32...."


In [6]:
date_start = pd.datetime(1800, 12, 31)
date_end = pd.datetime(2009, 12, 31)

date_scale = DateScale(min=date_start, max=date_end)

C:\Users\bhereth\AppData\Local\Temp\ipykernel_20668\1043072328.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  date_start = pd.datetime(1800, 12, 31)
C:\Users\bhereth\AppData\Local\Temp\ipykernel_20668\1043072328.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  date_end = pd.datetime(2009, 12, 31)


In [7]:
date_data = pd.date_range(start=date_start, end=date_end, freq="A", normalize=True)

In [8]:
country_name = "Angola"
data_name = "income"

x_data = data[data.name == country_name][data_name].values[0]


In [9]:
x_scale = LinearScale()

lines = Lines(x=date_data, y=x_data, scales={"x": date_scale, "y": x_scale})

ax_x = Axis(label="Year", scale=date_scale, num_ticks=10, tick_format="%Y")
ax_y = Axis(
    label=data_name.capitalize(), scale=x_scale, orientation="vertical", side="left"
)

figure = Figure(
    axes=[ax_x, ax_y],
    title=country_name,
    marks=[lines],
    animation_duration=500,
    layout={"max_height": "250px", "max_width": "400px"},
)
figure


Figure(animation_duration=500, axes=[Axis(label='Year', num_ticks=10, scale=DateScale(max=datetime.datetime(20…

In [10]:
def update_figure(country_name, data_name):
    lines.y = data[data.name == country_name][data_name].values[0]
    ax_y.label = data_name.capitalize()
    figure.title = country_name
    


In [11]:
country_name = "Benin"
data_name = "income"

update_figure(country_name, data_name)

In [12]:
country_name = "Angola"
data_name = "population"

update_figure(country_name, data_name)


In [13]:
with open("./countries.geo.json") as f:
    countries = json.load(f)

In [ ]:
m = Map(zoom=3)

geo = GeoJSON(
    data=countries,
    style={"fillColor": "white", "weight": 0.5},
    hover_style={"fillColor": "#1f77b4"},
    name="Countries",
)
m.add(geo)

m



Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

feature {'type': 'Feature', 'id': 'BRA', 'properties': {'name': 'Brazil', 'style': {'fillColor': 'white', 'weight': 0.5}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-57.625133, -30.216295], [-56.2909, -28.852761], [-55.162286, -27.881915], [-54.490725, -27.474757], [-53.648735, -26.923473], [-53.628349, -26.124865], [-54.13005, -25.547639], [-54.625291, -25.739255], [-54.428946, -25.162185], [-54.293476, -24.5708], [-54.29296, -24.021014], [-54.652834, -23.839578], [-55.027902, -24.001274], [-55.400747, -23.956935], [-55.517639, -23.571998], [-55.610683, -22.655619], [-55.797958, -22.35693], [-56.473317, -22.0863], [-56.88151, -22.282154], [-57.937156, -22.090176], [-57.870674, -20.732688], [-58.166392, -20.176701], [-57.853802, -19.969995], [-57.949997, -19.400004], [-57.676009, -18.96184], [-57.498371, -18.174188], [-57.734558, -17.552468], [-58.280804, -17.27171], [-58.388058, -16.877109], [-58.24122, -16.299573], [-60.15839, -16.258284], [-60.542966, -15.09391], [-60.251149

IndexError: index 0 is out of bounds for axis 0 with size 0

IndexError: index 0 is out of bounds for axis 0 with size 0

IndexError: index 0 is out of bounds for axis 0 with size 0

...

feature {'type': 'Feature', 'id': 'TCD', 'properties': {'name': 'Chad', 'style': {'fillColor': 'white', 'weight': 0.5}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[14.495787, 12.859396], [14.595781, 13.330427], [13.954477, 13.353449], [13.956699, 13.996691], [13.540394, 14.367134], [13.97217, 15.68437], [15.247731, 16.627306], [15.300441, 17.92795], [15.685741, 19.95718], [15.903247, 20.387619], [15.487148, 20.730415], [15.47106, 21.04845], [15.096888, 21.308519], [14.8513, 22.86295], [15.86085, 23.40972], [19.84926, 21.49509], [23.83766, 19.58047], [23.88689, 15.61084], [23.02459, 15.68072], [22.56795, 14.94429], [22.30351, 14.32682], [22.51202, 14.09318], [22.18329, 13.78648], [22.29658, 13.37232], [22.03759, 12.95546], [21.93681, 12.58818], [22.28801, 12.64605], [22.49762, 12.26024], [22.50869, 11.67936], [22.87622, 11.38461], [22.864165, 11.142395], [22.231129, 10.971889], [21.723822, 10.567056], [21.000868, 9.475985], [20.059685, 9.012706], [19.094008, 9.074847], [18.81201,

IndexError: index 0 is out of bounds for axis 0 with size 0

feature {'type': 'Feature', 'id': 'BRA', 'properties': {'name': 'Brazil', 'style': {'fillColor': 'white', 'weight': 0.5}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-57.625133, -30.216295], [-56.2909, -28.852761], [-55.162286, -27.881915], [-54.490725, -27.474757], [-53.648735, -26.923473], [-53.628349, -26.124865], [-54.13005, -25.547639], [-54.625291, -25.739255], [-54.428946, -25.162185], [-54.293476, -24.5708], [-54.29296, -24.021014], [-54.652834, -23.839578], [-55.027902, -24.001274], [-55.400747, -23.956935], [-55.517639, -23.571998], [-55.610683, -22.655619], [-55.797958, -22.35693], [-56.473317, -22.0863], [-56.88151, -22.282154], [-57.937156, -22.090176], [-57.870674, -20.732688], [-58.166392, -20.176701], [-57.853802, -19.969995], [-57.949997, -19.400004], [-57.676009, -18.96184], [-57.498371, -18.174188], [-57.734558, -17.552468], [-58.280804, -17.27171], [-58.388058, -16.877109], [-58.24122, -16.299573], [-60.15839, -16.258284], [-60.542966, -15.09391], [-60.251149

IndexError: index 0 is out of bounds for axis 0 with size 0

feature {'type': 'Feature', 'id': 'SEN', 'properties': {'name': 'Senegal', 'style': {'fillColor': 'white', 'weight': 0.5}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-16.713729, 13.594959], [-17.126107, 14.373516], [-17.625043, 14.729541], [-17.185173, 14.919477], [-16.700706, 15.621527], [-16.463098, 16.135036], [-16.12069, 16.455663], [-15.623666, 16.369337], [-15.135737, 16.587282], [-14.577348, 16.598264], [-14.099521, 16.304302], [-13.435738, 16.039383], [-12.830658, 15.303692], [-12.17075, 14.616834], [-12.124887, 13.994727], [-11.927716, 13.422075], [-11.553398, 13.141214], [-11.467899, 12.754519], [-11.513943, 12.442988], [-11.658301, 12.386583], [-12.203565, 12.465648], [-12.278599, 12.35444], [-12.499051, 12.33209], [-13.217818, 12.575874], [-13.700476, 12.586183], [-15.548477, 12.62817], [-15.816574, 12.515567], [-16.147717, 12.547762], [-16.677452, 12.384852], [-16.841525, 13.151394], [-15.931296, 13.130284], [-15.691001, 13.270353], [-15.511813, 13.27857], [-15.141

In [15]:
widget_control1 = WidgetControl(widget=figure, position="bottomright")

m.add(widget_control1)


def on_hover(event, feature, **kwargs):
    global country_name

    country_name = feature["properties"]["name"]
    update_figure(country_name, data_name)
    print("feature",feature)
geo.on_hover(on_hover)



In [16]:
dropdown = Dropdown(
    options=["income", "population", "lifeExpectancy"],
    value=data_name,
    description="Plotting:",
)


def on_click(change):
    global data_name

    data_name = change["new"]
    update_figure(country_name, data_name)


dropdown.observe(on_click, "value")

widget_control2 = WidgetControl(widget=dropdown, position="bottomleft")

m.add(widget_control2)


Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…